In [1]:
import netCDF4 as nc
import numpy as np
import glob, os

import mmw

bigf = 1e99

In [2]:
mmw_data = np.loadtxt("mmw/dat/combined.csv",delimiter=',',dtype=str)
mmw_table = {}
for r in mmw_data:
    mmw_table[r[0]] = float(r[1])

In [3]:
files = glob.glob("lv/dat/*.csv")
gases = [mmw.formula_from_path(f) for f in files]

In [4]:
# Find the name JANAF gives 'gas', based on number of atoms
def find_janaf_name(gas:str):

    target = mmw.count_atoms(gas)

    files = glob.glob("cp/dat/*.csv")
    for f in files:
        form  = mmw.formula_from_path(f)
        atoms = mmw.count_atoms(form)

        good = True  

        for k in atoms.keys():
            if k not in target.keys():
                good = False 
                break 

        for k in target.keys():
            if k not in atoms.keys():
                good = False 
                break  
            if target[k] != atoms[k]:
                good = False 
                break
        
        if good:
            return form 

    return ""


In [5]:
def compile(gas:str):
    print("Compiling %s"%gas)

    # JANAF alias 
    janaf = find_janaf_name(gas) 
    if len(janaf) == 0:
        print("    skipping (no JANAF data)")
        return 
    
    # create 
    fpath = "compiled/%s.ncdf"%gas
    if os.path.exists(fpath):
        os.remove(fpath)
    ds = nc.Dataset(fpath, 'w')

    # JANAF name 
    ds.createDimension('nchar', len(janaf))
    NC_janaf = ds.createVariable('JANAF', 'S1', ('nchar'))
    NC_janaf[:] = nc.stringtochar(np.array([janaf], 'S'))

    # MMW
    NC_mmw = ds.createVariable("mmw","f8")
    NC_mmw[:] = float(mmw_table[janaf])
    NC_mmw.units = "kg mol-1"

    # Load saturation data
    T_trip = 0.0
    T_crit = 0.0
    X_sat = [[0.0,bigf],[bigf,bigf]]
    if os.path.exists("sat/dat/%s_trip.csv"%gas):
        T_trip = float(np.loadtxt("sat/dat/%s_trip.csv"%gas))
        T_crit = float(np.loadtxt("sat/dat/%s_crit.csv"%gas))
        X_sat  = np.loadtxt("sat/dat/%s_sat.csv"%gas, delimiter=',').T
    else:
        print("    without saturation data")

    # Critical point
    NC_tcrit = ds.createVariable("T_crit","f8")
    NC_tcrit[:] = T_crit
    NC_tcrit.units = "K"

    # Triple point
    NC_ttrip = ds.createVariable("T_trip","f8")
    NC_ttrip[:] = T_trip
    NC_ttrip.units = "K"

    # Saturation curve 
    ds.createDimension("sat",len(X_sat[0]))
    #    temperatures
    NC_tsat = ds.createVariable("T_sat","f8","sat")
    NC_tsat[:] = X_sat[0][:]
    NC_tsat.units = "K"
    #    pressures
    NC_psat = ds.createVariable("P_sat","f8","sat")
    NC_psat[:] = X_sat[1][:]
    NC_psat.units = "Pa"
    
    # Latent heat 
    X_lat = np.loadtxt("lv/dat/%s.csv"%gas, delimiter=',').T
    ds.createDimension("lat",len(X_lat[0]))
    #    lookup temperatures 
    NC_dHt = ds.createVariable("deltaH_T","f8","lat")
    NC_dHt[:] = X_lat[0][:]
    NC_dHt.units = "K"
    #    enthalpy changes [J/kg]
    NC_dH = ds.createVariable("deltaH","f8","lat")
    NC_dH[:] = X_lat[1][:]
    NC_dH.units = "J kg-1"

    # Heat capacity
    X_cap = np.loadtxt("cp/dat/%s.csv"%janaf, delimiter=',').T
    ds.createDimension("cap",len(X_cap[0]))
    #    lookup temperatures 
    NC_cpt = ds.createVariable("Cp_T","f8","cap")
    NC_cpt[:] = X_cap[0][:]
    NC_cpt.units = "K"
    #   heat capacity
    NC_cp = ds.createVariable("Cp","f8","cap")
    NC_cp[:] = X_cap[1][:]
    NC_cp.units = "J mol-1 K-1"

    # done 
    ds.close()
    print("    ok")


In [6]:
for g in gases:
    compile(g)

Compiling CHN
    without saturation data
    ok
Compiling O2
    without saturation data
    ok
Compiling N2
    ok
Compiling SO2
    ok
Compiling C4H8
    skipping (no JANAF data)
Compiling NH3
    ok
Compiling H2
    ok
Compiling N2O
    ok
Compiling F2
    skipping (no JANAF data)
Compiling CO2
    ok
Compiling H2O
    ok
Compiling CO
    without saturation data
    ok
Compiling C2H4
    without saturation data
    ok
Compiling HCl
    skipping (no JANAF data)
Compiling C2H6
    skipping (no JANAF data)
Compiling CH4
    ok
